In [1]:
import os
import sys

spark_path = "C:\spark"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.4-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [2]:
sc=SparkContext(appName="pracaDypl", master="local[*]")

In [ ]:
## sprawdzenie czy działa spark

In [3]:
rdd = sc.textFile("C:\\spark\README.md")

In [4]:
rdd.count()

103

In [5]:
sc.startTime

1525002273277

## 1. Import danych. 

In [ ]:
# Dane, na których będziemy pracować to dump forum Stack Exchange. 
# Dane są dostępne pod adresem "https://archive.org/details/stackexchange".
# Poszczególne działy forum są udostępnione w postaci plików XML. 

# Do dalszej analizy wybrano dział: Datascience, plik: Posts. W pliku znjdują się szczegółowe informacje odnośnie wpisów
# na forum, poczynając od id, daty, powiązań z innymi wpisami, na treści wpisu kończąc. Celem jest wyciągnięcie samej treści 
# pytań.

## Metoda pierwsza

In [126]:
# bezpośrednio wczytujemy plik XML do Sparkowego RDD 
lines = sc.textFile("file:///Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
# # Czy ścieżkę dostępu da się zwinąć do zmiennej? a może sam url? jest jakiś problem ze znakami z XMLa, zamienia na papkę 

In [9]:
lines.take(6)

['<?xml version="1.0" encoding="utf-8"?>',
 '<posts>',
 '  <row Id="5" PostTypeId="1" CreationDate="2014-05-13T23:58:30.457" Score="8" ViewCount="412" Body="&lt;p&gt;I\'ve always been interested in machine learning, but I can\'t figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn\'t just use relative motion, because the obstacles move around, but I don\'t want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;" OwnerUserId="5" LastActivityDate="2014-05-14T00:36:31.077" Title="How can I do simple machine learning without hard-coding behavior?" Tags="&lt;machine-learning&gt;" AnswerCount="1" CommentCount="1" FavoriteCount="1" Closed

In [12]:
lines.count()

19216

In [ ]:
# Poza pierwszymi dwoma wierszami i ostatnim, jeden wiersz to komplet informacji na temat jednego pytania lub odpowiedzi 

In [ ]:
# Jak usunąć dwa pierwsze wiersze z RDD? filter? 

In [7]:
posts = lines.filter(lambda x: "row Id" in x)

In [13]:
posts.take(3)

['  <row Id="5" PostTypeId="1" CreationDate="2014-05-13T23:58:30.457" Score="8" ViewCount="412" Body="&lt;p&gt;I\'ve always been interested in machine learning, but I can\'t figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn\'t just use relative motion, because the obstacles move around, but I don\'t want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;" OwnerUserId="5" LastActivityDate="2014-05-14T00:36:31.077" Title="How can I do simple machine learning without hard-coding behavior?" Tags="&lt;machine-learning&gt;" AnswerCount="1" CommentCount="1" FavoriteCount="1" ClosedDate="2014-05-14T14:40:25.950" />',
 '  <row Id="7" Pos

In [14]:
# zniknęły trzy wiersze, a nie tylko pierwsze dwa... jeszcze pewnie na końcu coś wypadło
posts.count()


19213

In [ ]:
# Czyli mamy w dziale DataScience 19213 pytań i dopowiedzi.

In [ ]:
# Z tego miejsca skaczemy do Tomkowego pomysłu na dzielenie

## Metoda Druga

In [ ]:
# Drugi pomysł, to użycie pakietu w Pysparku do importu xml.

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [39]:
df = sqlContext.read.format('com.databricks.spark.xml').options(rowTag='user'). \
load("file:///Users/Spal/Desktop/Praca dyplomowa/XML testing/Users1.xml")

Py4JJavaError: An error occurred while calling o173.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.databricks.spark.xml. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:546)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:87)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:302)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:178)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:156)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.jav

In [37]:
sc.version # która wersja sparka, Scala jest w wersji 2.11.8

'2.2.1'

In [ ]:
# Sypie błędami i nie umiem tego przeskoczyć.

## Metoda trzecia.

In [ ]:
# Kolejny pomysł, to użycie pakietu spark xml bezpośrednio w scali. W notatniku wpisuję treść poleceń, które należy przekleić
# do cmd.

In [ ]:
# Należy uruchomić spark shell w trybie xml. Cyferki na końcu to wersja scali, która ma odpowiadać zainstalowanej.

In [ ]:
spark-shell --packages com.databricks:spark-xml_2.11:0.4.1

In [ ]:
# Następnie

In [ ]:
# Zaimportować sql context
import org.apache.spark.sql.SQLContext
# Dodac sql context
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
# Wczytanie pliku, utworzenie data framu, jest problem z prawidłowym podaniem ścieżki pliku
val df = sqlContext.read.format("com.databricks.spark.xml").option("rowTag", "user").load("file:///Users/Spal/Desktop/Praca dyplomowa/Dane/Users.xml")
val df4 = sqlContext.read.format("com.databricks.spark.xml").schema(customSchema).load("C://Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
val df5 = sqlContext.read.format("com.databricks.spark.xml").load("C://Users/Spal/Desktop/Praca dyplomowa/XML testing/Users.xml")
#podgląd 
df.show()

In [ ]:
# wychodzi pusta data frame bez informacji o błedzie... nie wiem co jest nie tak... format pliku xml trochę inny niż wymagany?
# brak zamknięcia </row> ? 
# Problem wynika z tego, że w dokumenatcji mamy "At the moment, rows containing self closing xml tags are not supported"...
# czyli XML z danymi jest akurat w takiej postaci, której nie rozumie spark xml

In [ ]:
# Znalazłem obejście - przekształcenie pliku XML do postaci zjadliwej dla spark xml
# trzeba utworzyć plik xsl, w którym definiujemy nową strukturę xml 
# w pliku xml trzeba zapisać odniesienie do xls
# po otwarciu za pomocą word zapisuje jako oddzielny plik xml
# trzeba go pozniej otworzyc w notatniku i wyciać odniesienei do xsla 
# i to już jest plik, który spark łyknie

In [16]:
## Metoda czwarta

In [ ]:
# bezpośrednio w Scali 

In [ ]:
# piszemy
import xml._
XML.loadFile("C://Users/Spal/Desktop/Praca dyplomowa/Dane/Posts.xml")
# i cały plik się wyświetli w konsoli, co mi z tego? nie mam pojęcia
# wyciąganie zawartości
res1 \ "row"
res2 \\"@Body" # to działa, ale czy to jest BIG DATA czy tylko programowanie w Scali?

# 2. Analiza zbioru. Proste transformacje.

## Metoda pierwsza cd.

In [ ]:
# Potrzebna metoda na wyfiltrowanie konkretnego fragmentu informacji z wiersza.

In [7]:
# Jak dobrać się do konkretnej porcji danych? Pierwszy pomysł: map, split i rozdzielenie spacja? źle...
fields = posts.map(lambda x: x.split(' '))

In [23]:
fields.take(3)

[['',
  '',
  '<row',
  'Id="5"',
  'PostTypeId="1"',
  'CreationDate="2014-05-13T23:58:30.457"',
  'Score="8"',
  'ViewCount="412"',
  'Body="&lt;p&gt;I\'ve',
  'always',
  'been',
  'interested',
  'in',
  'machine',
  'learning,',
  'but',
  'I',
  "can't",
  'figure',
  'out',
  'one',
  'thing',
  'about',
  'starting',
  'out',
  'with',
  'a',
  'simple',
  '&quot;Hello',
  'World&quot;',
  'example',
  '-',
  'how',
  'can',
  'I',
  'avoid',
  'hard-coding',
  'behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For',
  'example,',
  'if',
  'I',
  'wanted',
  'to',
  '&quot;teach&quot;',
  'a',
  'bot',
  'how',
  'to',
  'avoid',
  'randomly',
  'placed',
  'obstacles,',
  'I',
  "couldn't",
  'just',
  'use',
  'relative',
  'motion,',
  'because',
  'the',
  'obstacles',
  'move',
  'around,',
  'but',
  'I',
  "don't",
  'want',
  'to',
  'hard',
  'code,',
  'say,',
  'distance,',
  'because',
  'that',
  'ruins',
  'the',
  'whole',
  'point',
  'of',
  'machine',
  'learning.&lt;/p&

In [ ]:
# Porcja danych ma zawsze postać XX="....". Dla danego XX chciałbym wydobyć fragment znajdujący się pomiędzy " ".

In [ ]:
# czyli jakoś tak, pożniej odliczyć na którym miejscu jest interesujące nas pole i ognia

In [24]:
fields = posts.map(lambda x: x.split('"'))

In [27]:
fields.take(1)

[['  <row Id=',
  '5',
  ' PostTypeId=',
  '1',
  ' CreationDate=',
  '2014-05-13T23:58:30.457',
  ' Score=',
  '8',
  ' ViewCount=',
  '412',
  ' Body=',
  "&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",
  ' OwnerUserId=',
  '5',
  ' LastActivityDate=',
  '2014-05-14T00:36:31.077',
  ' Title=',
  'How can I do simple machine learning without hard-coding behavior?',
  ' Tags=',
  '&lt;machine-learning&gt;',
  ' AnswerCount=',


In [ ]:
# Tu na szczęście przyszła odsiecz.

In [8]:
def cleaningRDD(RDDIn):
    def cleaning(listIn):
        listOut=[]
        for x in listIn:
            if listIn.index(x)%2==0:
                listOut.append(x.replace(' ','').replace('<','').replace('=',''))
            else:
                listOut.append(x)
        return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
    return RDDIn.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning)

In [127]:
cleared=cleaningRDD(lines)

In [29]:
cleared.take(5)

[{'AnswerCount': '1',
  'Body': "&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",
  'ClosedDate': '2014-05-14T14:40:25.950',
  'CommentCount': '1',
  'CreationDate': '2014-05-13T23:58:30.457',
  'FavoriteCount': '1',
  'LastActivityDate': '2014-05-14T00:36:31.077',
  'OwnerUserId': '5',
  'PostTypeId': '1',
  'Score': '8',
  'Tags': '&lt;machine-learning&gt;',
  'Title': 'How can I do simple machine learning without hard-coding 

In [13]:
cleared.map(lambda x: x['Body']).take(3)

["&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",
 "&lt;p&gt;As a researcher and instructor, I'm looking for open-source books (or similar materials) that provide a relatively thorough overview of data science from an applied perspective. To be clear, I'm especially interested in a thorough overview that provides material suitable for a college-level course, not particular pieces or papers.&lt;/p&gt;&#xA;",
 "&lt;p&gt;Not sure 

In [ ]:
# kolejna zagadka, to wybranie wszystkich komentarzy, które są pytaniem. Czyli 'PostTypeId': '1',

In [17]:
cleared.filter(lambda x: x=="'PostTypeId': '1'").take(2)

SyntaxError: lambda cannot contain assignment (<ipython-input-17-7e0d3340baa2>, line 1)

In [21]:
for i in cleared.take(2): print(i)

{'rowId': '5', 'ViewCount': '412', 'Title': 'How can I do simple machine learning without hard-coding behavior?', 'AnswerCount': '1', 'ClosedDate': '2014-05-14T14:40:25.950', 'CreationDate': '2014-05-13T23:58:30.457', 'FavoriteCount': '1', 'CommentCount': '1', 'Body': "&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;", 'LastActivityDate': '2014-05-14T00:36:31.077', 'Score': '8', 'OwnerUserId': '5', 'PostTypeId': '1', 'Tags': '&lt;

In [28]:
cleared[2] # nie można 

TypeError: 'PipelinedRDD' object does not support indexing

In [29]:
zebrane = cleared.collect()

In [33]:
zebrane[5] # po collect już tak, ale collect ściąga rdd na driver, to raczej bez sensu ruch

{'AnswerCount': '0',
 'Body': '&lt;p&gt;In which situations would one system be preferred over the other? What are the relative advantages and disadvantages of relational databases versus non-relational databases?&lt;/p&gt;&#xA;',
 'ClosedDate': '2014-05-14T07:41:49.437',
 'CommentCount': '1',
 'CreationDate': '2014-05-14T01:41:23.110',
 'LastActivityDate': '2014-05-14T01:41:23.110',
 'OwnerUserId': '64',
 'PostTypeId': '1',
 'Score': '2',
 'Tags': '&lt;databases&gt;',
 'Title': 'What are the advantages and disadvantages of SQL versus NoSQL in data science?',
 'ViewCount': '494',
 'rowId': '15'}

In [31]:
zebrane[2]["Score"]

'5'

In [36]:
zebrane["Score"==5] # tak nie działa

{'AnswerCount': '1',
 'Body': "&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",
 'ClosedDate': '2014-05-14T14:40:25.950',
 'CommentCount': '1',
 'CreationDate': '2014-05-13T23:58:30.457',
 'FavoriteCount': '1',
 'LastActivityDate': '2014-05-14T00:36:31.077',
 'OwnerUserId': '5',
 'PostTypeId': '1',
 'Score': '8',
 'Tags': '&lt;machine-learning&gt;',
 'Title': 'How can I do simple machine learning without hard-coding behavior?',


In [ ]:
# Utworzenie data frame

In [11]:
df=cleared.toDF(['AnswerCount','Body','ClosedDate','CommentCount','CreationDate','FavoriteCount'])

C:\spark/python\pyspark\sql\session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [10]:
spark = SparkSession.builder \
    .appName('DataFrame_1') \
    .master('local[*]') \
    .getOrCreate()

In [128]:
df=spark.createDataFrame(cleared) ##czyli że co?, płacze ale działa

C:\spark/python\pyspark\sql\session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [21]:
print cleared.first(1)


SyntaxError: invalid syntax (<ipython-input-21-aea77fc92f24>, line 1)

In [12]:
type(cleared)

pyspark.rdd.PipelinedRDD

In [22]:
df.show()


+-----------+--------------------+--------------------+------------+--------------------+-------------+--------------------+-----------+----------+-----+--------------------+--------------------+---------+-----+
|AnswerCount|                Body|          ClosedDate|CommentCount|        CreationDate|FavoriteCount|    LastActivityDate|OwnerUserId|PostTypeId|Score|                Tags|               Title|ViewCount|rowId|
+-----------+--------------------+--------------------+------------+--------------------+-------------+--------------------+-----------+----------+-----+--------------------+--------------------+---------+-----+
|          1|&lt;p&gt;I've alw...|2014-05-14T14:40:...|           1|2014-05-13T23:58:...|            1|2014-05-14T00:36:...|          5|         1|    8|&lt;machine-learn...|How can I do simp...|      412|    5|
|          3|&lt;p&gt;As a res...|2014-05-14T08:40:...|           4|2014-05-14T00:11:...|            1|2014-05-16T13:45:...|         36|         1|    4

In [23]:
df.dtypes

[('AnswerCount', 'string'),
 ('Body', 'string'),
 ('ClosedDate', 'string'),
 ('CommentCount', 'string'),
 ('CreationDate', 'string'),
 ('FavoriteCount', 'string'),
 ('LastActivityDate', 'string'),
 ('OwnerUserId', 'string'),
 ('PostTypeId', 'string'),
 ('Score', 'string'),
 ('Tags', 'string'),
 ('Title', 'string'),
 ('ViewCount', 'string'),
 ('rowId', 'string')]

In [19]:
df.show(5)

+-----------+--------------------+--------------------+------------+--------------------+-------------+--------------------+-----------+----------+-----+--------------------+--------------------+---------+-----+
|AnswerCount|                Body|          ClosedDate|CommentCount|        CreationDate|FavoriteCount|    LastActivityDate|OwnerUserId|PostTypeId|Score|                Tags|               Title|ViewCount|rowId|
+-----------+--------------------+--------------------+------------+--------------------+-------------+--------------------+-----------+----------+-----+--------------------+--------------------+---------+-----+
|          1|&lt;p&gt;I've alw...|2014-05-14T14:40:...|           1|2014-05-13T23:58:...|            1|2014-05-14T00:36:...|          5|         1|    8|&lt;machine-learn...|How can I do simp...|      412|    5|
|          3|&lt;p&gt;As a res...|2014-05-14T08:40:...|           4|2014-05-14T00:11:...|            1|2014-05-16T13:45:...|         36|         1|    4

In [30]:
df.\
select('Body').\
where(df.PostTypeId.like('%1%')).\
show(500)

+--------------------+
|                Body|
+--------------------+
|&lt;p&gt;I've alw...|
|&lt;p&gt;As a res...|
|&lt;p&gt;I am sur...|
|&lt;p&gt;In which...|
|&lt;p&gt;I use &l...|
|&lt;p&gt;Lots of ...|
|&lt;p&gt;we creat...|
|&lt;p&gt;My data ...|
|&lt;p&gt;I have a...|
|&lt;p&gt;In worki...|
|&lt;p&gt;I heard ...|
|&lt;p&gt;R has ma...|
|&lt;p&gt;I have a...|
|&lt;p&gt;From my ...|
|&lt;p&gt;In revie...|
|&lt;p&gt;Logic of...|
|&lt;p&gt;First, t...|
|&lt;p&gt;What are...|
|&lt;p&gt;If small...|
|&lt;p&gt;&lt;em&g...|
|&lt;p&gt;&lt;stro...|
|&lt;p&gt;What is(...|
|&lt;p&gt;Given we...|
|&lt;p&gt;For exam...|
|&lt;p&gt;While bu...|
|&lt;p&gt;What is ...|
|&lt;p&gt;Assume t...|
|&lt;p&gt;Consider...|
|&lt;p&gt;When a r...|
|&lt;p&gt;If I hav...|
|&lt;p&gt;I have a...|
|&lt;p&gt;I want l...|
|&lt;p&gt;&lt;a hr...|
|&lt;p&gt;&lt;a hr...|
|&lt;p&gt;From wik...|
|&lt;p&gt;In our c...|
|&lt;p&gt;Any smal...|
|&lt;p&gt;As we al...|
|&lt;p&gt;One of t...|
|&lt;p&gt;I see a ...|
|&lt;p&gt;A

In [129]:
samTekst=df.\
select('Body').\
where(df.PostTypeId.like('%1%'))

In [33]:
samTekst.take(5) ### lo matko

[Row(Body="&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;"),
 Row(Body="&lt;p&gt;As a researcher and instructor, I'm looking for open-source books (or similar materials) that provide a relatively thorough overview of data science from an applied perspective. To be clear, I'm especially interested in a thorough overview that provides material suitable for a college-level course, not particular pieces or papers.&lt;/p&gt;&#xA;"),


In [38]:
df.printSchema()

root
 |-- AnswerCount: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- CommentCount: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- FavoriteCount: string (nullable = true)
 |-- LastActivityDate: string (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- PostTypeId: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: string (nullable = true)
 |-- rowId: string (nullable = true)



In [14]:
# zeby robić kwerendy sql trzeba stworzyć temporary view
df.createOrReplaceTempView("df")

In [44]:
df.select("Body").filter("PostTypeId=1").show(100)

+--------------------+
|                Body|
+--------------------+
|&lt;p&gt;I've alw...|
|&lt;p&gt;As a res...|
|&lt;p&gt;I am sur...|
|&lt;p&gt;In which...|
|&lt;p&gt;I use &l...|
|&lt;p&gt;Lots of ...|
|&lt;p&gt;we creat...|
|&lt;p&gt;My data ...|
|&lt;p&gt;I have a...|
|&lt;p&gt;In worki...|
|&lt;p&gt;I heard ...|
|&lt;p&gt;R has ma...|
|&lt;p&gt;I have a...|
|&lt;p&gt;From my ...|
|&lt;p&gt;In revie...|
|&lt;p&gt;Logic of...|
|&lt;p&gt;First, t...|
|&lt;p&gt;What are...|
|&lt;p&gt;If small...|
|&lt;p&gt;&lt;em&g...|
|&lt;p&gt;&lt;stro...|
|&lt;p&gt;What is(...|
|&lt;p&gt;Given we...|
|&lt;p&gt;For exam...|
|&lt;p&gt;While bu...|
|&lt;p&gt;What is ...|
|&lt;p&gt;Assume t...|
|&lt;p&gt;Consider...|
|&lt;p&gt;When a r...|
|&lt;p&gt;If I hav...|
|&lt;p&gt;I have a...|
|&lt;p&gt;I want l...|
|&lt;p&gt;&lt;a hr...|
|&lt;p&gt;&lt;a hr...|
|&lt;p&gt;From wik...|
|&lt;p&gt;In our c...|
|&lt;p&gt;Any smal...|
|&lt;p&gt;As we al...|
|&lt;p&gt;One of t...|
|&lt;p&gt;I see a ...|
|&lt;p&gt;A

In [10]:
samTekst.cache()

DataFrame[Body: string]

In [13]:
samTekst.show(1)

+--------------------+
|                Body|
+--------------------+
|&lt;p&gt;I've alw...|
+--------------------+
only showing top 1 row



In [55]:
type(samTekst)

pyspark.sql.dataframe.DataFrame

In [15]:
type(samTekst.rdd)

pyspark.rdd.RDD

In [16]:
samTekst.rdd.collect()

[Row(Body="&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;"),
 Row(Body="&lt;p&gt;As a researcher and instructor, I'm looking for open-source books (or similar materials) that provide a relatively thorough overview of data science from an applied perspective. To be clear, I'm especially interested in a thorough overview that provides material suitable for a college-level course, not particular pieces or papers.&lt;/p&gt;&#xA;"),


In [18]:
samTekst.rdd.map(tuple).collect()

[("&lt;p&gt;I've always been interested in machine learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relative motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;",),
 ("&lt;p&gt;As a researcher and instructor, I'm looking for open-source books (or similar materials) that provide a relatively thorough overview of data science from an applied perspective. To be clear, I'm especially interested in a thorough overview that provides material suitable for a college-level course, not particular pieces or papers.&lt;/p&gt;&#xA;",),
 ('&lt;p&gt;I 

In [15]:
samTekstrdd=samTekst.rdd.map(tuple)

In [53]:
type(samTekstrdd)

pyspark.rdd.PipelinedRDD

In [ ]:
https://stackoverflow.com/questions/35769489/adding-the-resulting-tfidf-calculation-to-the-dataframe-of-the-original-document

In [16]:
df1 = (samTekst
  .rdd
  .map(lambda x : (x, x.Body.split(" ")))
  .toDF()
  )

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 3, localhost, executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.org$apache$spark$api$python$PythonRDD$$write$1(PythonRDD.scala:506)
	at org.apache.spark.api.python.PythonRDD$$anonfun$writeIteratorToStream$1.apply(PythonRDD.scala:518)
	at org.apache.spark.api.

In [ ]:
## Transformers

In [17]:
import pyspark.ml.feature as ft

In [160]:
tokenizer = ft.RegexTokenizer( #źle działa :( 
    inputCol='Body', 
    outputCol='Body_arr', 
    pattern='\s+|[;,.\"?/&-)$0123456789:(]')

In [178]:
tokenizer = ft.RegexTokenizer( 
    inputCol='Body', 
    outputCol='Body_arr', 
    pattern='\\W+|[;,.\"?/&-)$0123456789:(]')

In [172]:
samTekst.printSchema()

root
 |-- Body: string (nullable = true)



In [179]:
tok = tokenizer \
    .transform(samTekst) \
    .select('Body_arr') 

tok.take(500)

[Row(Body_arr=['lt', 'p', 'gt', 'i', 've', 'always', 'been', 'interested', 'in', 'machine', 'learning', 'but', 'i', 'can', 't', 'figure', 'out', 'one', 'thing', 'about', 'starting', 'out', 'with', 'a', 'simple', 'quot', 'hello', 'world', 'quot', 'example', 'how', 'can', 'i', 'avoid', 'hard', 'coding', 'behavior', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'for', 'example', 'if', 'i', 'wanted', 'to', 'quot', 'teach', 'quot', 'a', 'bot', 'how', 'to', 'avoid', 'randomly', 'placed', 'obstacles', 'i', 'couldn', 't', 'just', 'use', 'relative', 'motion', 'because', 'the', 'obstacles', 'move', 'around', 'but', 'i', 'don', 't', 'want', 'to', 'hard', 'code', 'say', 'distance', 'because', 'that', 'ruins', 'the', 'whole', 'point', 'of', 'machine', 'learning', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'obviously', 'randomly', 'generating', 'code', 'would', 'be', 'impractical', 'so', 'how', 'could', 'i', 'do', 'this', 'lt', 'p', 'gt', 'xa']),
 Row(Body_arr=['lt', 'p', 'gt', 'as', 'a', 'resear

In [180]:
tok.printSchema()

root
 |-- Body_arr: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [40]:
#https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35
#https://spark.apache.org/docs/latest/ml-features.html

In [20]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer

In [181]:
add_stopwords = ["http","https","the", "lt", "p","gt", "#xa", "quot",  "href"] 

In [163]:
# z parametrem setStopWords usuwa słowa zdefiniowane w powyższej liście 
stopwordsRemover = StopWordsRemover(inputCol="Body_arr", outputCol="filtered").setStopWords(add_stopwords)


In [182]:
tok1=stopwordsRemover.transform(tok)
tok1.take(1)

[Row(Body_arr=['lt', 'p', 'gt', 'i', 've', 'always', 'been', 'interested', 'in', 'machine', 'learning', 'but', 'i', 'can', 't', 'figure', 'out', 'one', 'thing', 'about', 'starting', 'out', 'with', 'a', 'simple', 'quot', 'hello', 'world', 'quot', 'example', 'how', 'can', 'i', 'avoid', 'hard', 'coding', 'behavior', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'for', 'example', 'if', 'i', 'wanted', 'to', 'quot', 'teach', 'quot', 'a', 'bot', 'how', 'to', 'avoid', 'randomly', 'placed', 'obstacles', 'i', 'couldn', 't', 'just', 'use', 'relative', 'motion', 'because', 'the', 'obstacles', 'move', 'around', 'but', 'i', 'don', 't', 'want', 'to', 'hard', 'code', 'say', 'distance', 'because', 'that', 'ruins', 'the', 'whole', 'point', 'of', 'machine', 'learning', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'obviously', 'randomly', 'generating', 'code', 'would', 'be', 'impractical', 'so', 'how', 'could', 'i', 'do', 'this', 'lt', 'p', 'gt', 'xa'], filtered=['i', 've', 'always', 'been', 'interested'

In [ ]:
#bez parametru usuwa słowa, które ma zdefiniowane od kuchni

In [146]:
stopwordsRemover2 = StopWordsRemover(inputCol="filtered", outputCol="filtered1")

In [183]:
tok2=stopwordsRemover2.transform(tok1)
tok2.take(1)

[Row(Body_arr=['lt', 'p', 'gt', 'i', 've', 'always', 'been', 'interested', 'in', 'machine', 'learning', 'but', 'i', 'can', 't', 'figure', 'out', 'one', 'thing', 'about', 'starting', 'out', 'with', 'a', 'simple', 'quot', 'hello', 'world', 'quot', 'example', 'how', 'can', 'i', 'avoid', 'hard', 'coding', 'behavior', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'for', 'example', 'if', 'i', 'wanted', 'to', 'quot', 'teach', 'quot', 'a', 'bot', 'how', 'to', 'avoid', 'randomly', 'placed', 'obstacles', 'i', 'couldn', 't', 'just', 'use', 'relative', 'motion', 'because', 'the', 'obstacles', 'move', 'around', 'but', 'i', 'don', 't', 'want', 'to', 'hard', 'code', 'say', 'distance', 'because', 'that', 'ruins', 'the', 'whole', 'point', 'of', 'machine', 'learning', 'lt', 'p', 'gt', 'xa', 'xa', 'lt', 'p', 'gt', 'obviously', 'randomly', 'generating', 'code', 'would', 'be', 'impractical', 'so', 'how', 'could', 'i', 'do', 'this', 'lt', 'p', 'gt', 'xa'], filtered=['i', 've', 'always', 'been', 'interested'

In [166]:
tok2.select("filtered1").take(1)

[Row(filtered1=['ve', 'always', 'interested', 'machine', 'learning', 'figure', 'one', 'thing', 'starting', 'simple', 'hello', 'world', 'example', 'avoid', 'hard-coding', 'behavior', 'example', 'wanted', 'teach', 'bot', 'avoid', 'randomly', 'placed', 'obstacles', 'couldn', 'use', 'relative', 'motion', 'obstacles', 'move', 'around', 'want', 'hard', 'code', 'say', 'distance', 'ruins', 'whole', 'point', 'machine', 'learning', 'obviously', 'randomly', 'generating', 'code', 'impractical'])]

In [149]:
tok2.take(1)

[Row(Body_arr=["&lt;p&gt;i've", 'always', 'been', 'interested', 'in', 'machine', 'learning,', 'but', 'i', "can't", 'figure', 'out', 'one', 'thing', 'about', 'starting', 'out', 'with', 'a', 'simple', '&quot;hello', 'world&quot;', 'example', '-', 'how', 'can', 'i', 'avoid', 'hard-coding', 'behavior?&lt;/p&gt;&#xa;&#xa;&lt;p&gt;for', 'example,', 'if', 'i', 'wanted', 'to', '&quot;teach&quot;', 'a', 'bot', 'how', 'to', 'avoid', 'randomly', 'placed', 'obstacles,', 'i', "couldn't", 'just', 'use', 'relative', 'motion,', 'because', 'the', 'obstacles', 'move', 'around,', 'but', 'i', "don't", 'want', 'to', 'hard', 'code,', 'say,', 'distance,', 'because', 'that', 'ruins', 'the', 'whole', 'point', 'of', 'machine', 'learning.&lt;/p&gt;&#xa;&#xa;&lt;p&gt;obviously,', 'randomly', 'generating', 'code', 'would', 'be', 'impractical,', 'so', 'how', 'could', 'i', 'do', 'this?&lt;/p&gt;&#xa;'], filtered=["&lt;p&gt;i've", 'always', 'been', 'interested', 'in', 'machine', 'learning,', 'but', 'i', "can't", 'fig

In [184]:
tok3 = tok2.select("filtered1")

In [189]:
tok3.show(50)

AttributeError: 'builtin_function_or_method' object has no attribute '_get_object_id'

In [185]:
tok3.printSchema()

root
 |-- filtered1: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [170]:
tok3.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# teraz robimy df, w którym w pierszej kolumnie będzie klucz, a w drugiej odfiltrowane słowa

In [28]:
from pyspark.sql.functions import lit

In [35]:
df1_withkey = tok3.withColumn("key", lit(1))

In [36]:
df1_withkey.show(2)

+--------------------+---+
|           filtered1|key|
+--------------------+---+
|[always, interest...|  1|
|[researcher, inst...|  1|
+--------------------+---+
only showing top 2 rows



In [38]:
df1_withkey = df1_withkey["key", "filtered1"]

In [118]:
df1_withkey.show()

+---+--------------------+
|key|           filtered1|
+---+--------------------+
|  1|[always, interest...|
|  1|[researcher, inst...|
|  1|[sure, data, scie...|
|  1|[situations, one,...|
|  1|[use, href=, http...|
|  1|[lots, people, us...|
|  1|[created, social,...|
|  1|[data, set, conta...|
|  1|[bunch, customer,...|
|  1|[working, explora...|
|  1|[heard, many, too...|
|  1|[r, many, librari...|
|  1|[r, script, gener...|
|  1|[limited, dabblin...|
|  1|[reviewing, “, hr...|
|  1|[logic, often, st...|
|  1|[first, think, wo...|
|  1|[data, conditions...|
|  1|[small, values, p...|
|  1|[em, (note:, pull...|
+---+--------------------+
only showing top 20 rows



In [116]:
df1_withkey.printSchema() # czemu jest ten string?

root
 |-- key: integer (nullable = false)
 |-- filtered1: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [117]:
df1_withkey["element"]

AnalysisException: 'Cannot resolve column name "element" among (key, filtered1);'

In [ ]:
# ta sama procedura dla drugiej paczki postów z innego forum, przydzielamy inny klucz (lepiej byłoby utworzyć kolumnę z kluczem
# na wcześniejszym etapie i wszystkie czysczenia robić na połączonej tabeli)

In [40]:
lines = sc.textFile("file:///Users/Spal/Desktop/Praca dyplomowa/Dane1/Posts.xml")

In [41]:
cleared=cleaningRDD(lines)

In [44]:
df=spark.createDataFrame(cleared)

C:\spark/python\pyspark\sql\session.py:356: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [45]:
samTekst=df.\
select('Body').\
where(df.PostTypeId.like('%1%'))

In [65]:
tok = tokenizer \
    .transform(samTekst) \
    .select('Body_arr') 

tok.take(1)

[Row(Body_arr=['lt', 'p', 'gt', 'if', 'a', 'currency', 'can', 'be', 'worth', 'too', 'little', 'e', 'g', 'needing', 'to', 'buy', 'a', 'loaf', 'of', 'bread', 'and', 'worth', 'too', 'much', 'e', 'g', 'being', 'able', 'to', 'buy', 'a', 'loaf', 'of', 'bread', 'for', 'why', 'isn', 't', 'there', 'an', 'quot', 'ideal', 'value', 'quot', 'a', 'point', 'range', 'or', 'a', 'shifting', 'set', 'of', 'priorities', 'based', 'on', 'market', 'conditions', 'that', 'a', 'currency', 'could', 'be', 'fixed', 'or', 'drawn', 'toward', 'lt', 'p', 'gt', '#xa', '#xa', 'lt', 'p', 'gt', 'many', 'everyday', 'transactions', 'are', 'still', 'done', 'in', 'cash', 'requiring', 'mental', 'calculations', 'from', 'the', 'transacting', 'parties', 'too', 'large', 'numbers', 'and', 'too', 'small', 'numbers', 'create', 'transaction', 'costs', 'that', 'at', 'least', 'for', 'the', 'extremes', 'cannot', 'be', 'ignored', 'so', 'conceivably', 'some', 'quot', 'middle', 'quot', 'point', 'could', 'be', 'considered', 'quot', 'ideal', '

In [66]:
tok1=stopwordsRemover.transform(tok)
tok1.show(1)

+--------------------+--------------------+
|            Body_arr|            filtered|
+--------------------+--------------------+
|[lt, p, gt, if, a...|[if, a, currency,...|
+--------------------+--------------------+
only showing top 1 row



In [67]:
tok2=stopwordsRemover2.transform(tok1)
tok2.show(1)

+--------------------+--------------------+--------------------+
|            Body_arr|            filtered|           filtered1|
+--------------------+--------------------+--------------------+
|[lt, p, gt, if, a...|[if, a, currency,...|[currency, worth,...|
+--------------------+--------------------+--------------------+
only showing top 1 row



In [68]:
tok2.take(1)

[Row(Body_arr=['lt', 'p', 'gt', 'if', 'a', 'currency', 'can', 'be', 'worth', 'too', 'little', 'e', 'g', 'needing', 'to', 'buy', 'a', 'loaf', 'of', 'bread', 'and', 'worth', 'too', 'much', 'e', 'g', 'being', 'able', 'to', 'buy', 'a', 'loaf', 'of', 'bread', 'for', 'why', 'isn', 't', 'there', 'an', 'quot', 'ideal', 'value', 'quot', 'a', 'point', 'range', 'or', 'a', 'shifting', 'set', 'of', 'priorities', 'based', 'on', 'market', 'conditions', 'that', 'a', 'currency', 'could', 'be', 'fixed', 'or', 'drawn', 'toward', 'lt', 'p', 'gt', '#xa', '#xa', 'lt', 'p', 'gt', 'many', 'everyday', 'transactions', 'are', 'still', 'done', 'in', 'cash', 'requiring', 'mental', 'calculations', 'from', 'the', 'transacting', 'parties', 'too', 'large', 'numbers', 'and', 'too', 'small', 'numbers', 'create', 'transaction', 'costs', 'that', 'at', 'least', 'for', 'the', 'extremes', 'cannot', 'be', 'ignored', 'so', 'conceivably', 'some', 'quot', 'middle', 'quot', 'point', 'could', 'be', 'considered', 'quot', 'ideal', '

In [69]:
tok3 = tok2.select("filtered1")

In [73]:
df2_withkey = tok3.withColumn("key", lit(2))

In [74]:
df2_withkey = df2_withkey["key", "filtered1"]

In [75]:
df2_withkey.show(1)

+---+--------------------+
|key|           filtered1|
+---+--------------------+
|  2|[currency, worth,...|
+---+--------------------+
only showing top 1 row



In [82]:
df2_withkey.count()

6041

In [115]:
df2_withkey.printSchema()

root
 |-- key: integer (nullable = false)
 |-- filtered1: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
# i łączymy dwa data framy w jeden

In [79]:
fin_df = df1_withkey.union(df2_withkey)

In [81]:
fin_df.count()

14653

In [100]:
fin_df.show(2, False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key|filtered1                                                                                                                                                                                                                                                                                                                                                                        |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [114]:
fin_df.printSchema()

root
 |-- key: integer (nullable = false)
 |-- filtered1: array (nullable = true)
 |    |-- element: string (containsNull = true)



# TF IDF Model

In [ ]:
# teraz rozbudowujemy nasz df o kolumny tf i  idf

In [101]:
from pyspark.ml.feature import HashingTF as MLHashingTF
from pyspark.ml.feature import IDF as MLIDF
from pyspark.sql.types import DoubleType 

In [102]:
htf = MLHashingTF(inputCol = "filtered1", outputCol = "tf")

In [103]:
tf = htf.transform(fin_df)

In [104]:
tf.show(n=1, truncate=False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|key|filtered1                                                                                                                                                                                                             

In [105]:
tf.take(1)

[Row(key=1, filtered1=['always', 'interested', 'machine', 'learning', 'figure', 'one', 'thing', 'starting', 'simple', 'hello', 'world', 'example', 'avoid', 'hard', 'coding', 'behavior', 'example', 'wanted', 'teach', 'bot', 'avoid', 'randomly', 'placed', 'obstacles', 'use', 'relative', 'motion', 'obstacles', 'move', 'around', 'want', 'hard', 'code', 'say', 'distance', 'ruins', 'whole', 'point', 'machine', 'learning', 'obviously', 'randomly', 'generating', 'code', 'impractical'], tf=SparseVector(262144, {1836: 2.0, 2437: 2.0, 14072: 1.0, 24610: 2.0, 29238: 1.0, 43223: 1.0, 45245: 2.0, 53778: 1.0, 65212: 1.0, 78329: 1.0, 93206: 1.0, 93284: 2.0, 96550: 1.0, 107499: 1.0, 116873: 1.0, 118651: 2.0, 124189: 1.0, 125372: 1.0, 155074: 1.0, 159066: 1.0, 163984: 2.0, 165160: 1.0, 169195: 1.0, 175098: 1.0, 181726: 1.0, 190256: 1.0, 190355: 1.0, 192823: 1.0, 204376: 1.0, 217228: 1.0, 230591: 1.0, 235528: 1.0, 236821: 1.0, 242532: 1.0, 255376: 2.0, 256803: 1.0, 258463: 1.0}))]

In [108]:
tf.cache()

DataFrame[key: int, filtered1: array<string>, tf: vector]

In [113]:
tf.printSchema()

root
 |-- key: integer (nullable = false)
 |-- filtered1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)



In [111]:
 idf = MLIDF().fit(tf)

Py4JJavaError: An error occurred while calling o918.fit.
: java.util.NoSuchElementException: Failed to find a default value for inputCol
	at org.apache.spark.ml.param.Params$$anonfun$getOrDefault$2.apply(params.scala:652)
	at org.apache.spark.ml.param.Params$$anonfun$getOrDefault$2.apply(params.scala:652)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.ml.param.Params$class.getOrDefault(params.scala:651)
	at org.apache.spark.ml.PipelineStage.getOrDefault(Pipeline.scala:42)
	at org.apache.spark.ml.param.Params$class.$(params.scala:658)
	at org.apache.spark.ml.PipelineStage.$(Pipeline.scala:42)
	at org.apache.spark.ml.feature.IDFBase$class.validateAndTransformSchema(IDF.scala:59)
	at org.apache.spark.ml.feature.IDF.validateAndTransformSchema(IDF.scala:68)
	at org.apache.spark.ml.feature.IDF.transformSchema(IDF.scala:98)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.IDF.fit(IDF.scala:88)
	at org.apac

Py4JJavaError: An error occurred while calling o778.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 65, localhost, executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1517)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1505)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1504)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1504)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:814)
	at 

In [ ]:
tfidf.show(2) 

In [ ]:
tfidf.take(1)